# Database Tables


* Merging Graduation Rate and Report Card tables
	
	Graduation Rate		Report Card
	AGGREGATION_CODE   ==	ENTITY_CD


# Report Card Database Tables
* Combined state/federal expenditure per student by school and county: Expenditures per Pupil
	- PER_FED_STATE_LOCAL_EXP: combines state and federal
	- PUPIL_COUNT_TOT
	- YEAR
	- ENTITY_NAME
	- ENTITY_CD

* Staff inexperience or out of certification by school and county: Staff Qualifications or Staff
	- PER_TEACH_INEXP
	- PER_PRINC_INEXP
	- PER_OUT_CERT
	- ENTITY_NAME
	- ENTITY_CD

* Subgroup distributions: ACC HS Graduation Rate
	- SUBGROUP_NAME: ethnicity, disabilities, economically disadvantaged, English language learners.
	- COHORT_COUNT
	- GRAD_RATE
	- ENTITY_CD
	- ENTITY_NAME

* Annual EM ELA/Math/Science Proficiency: Annual EM ELA, Annual EM MATH, Annual EM SCIENCE
	- PER_PROF (Shows most distinction)
	- TOTAL_SCALE_SCORES
	- MEAN_SCORES
	- SUBGROUP_NAME
	- YEAR
	- ASSESSMENT_NAME
	- ENTITY_NAME
	- ENTITY_CD

* Graduation rates: ACC HS Graduation Rate
	- GRAD_RATE
	- COHORT_COUNT
	- YEAR
	- SUBGROUP_NAME
	- ENTITY_NAME
	- ENTITY_CD


In [1]:
import sys, subprocess
from io import StringIO
import pandas as pd
from importlib import reload
# import mdb_pandas_api

verbose = False
# reload(mdb_pandas_api)

In [2]:
# Read mdb into a dictionary of pandas tables
def mdb_to_pandas(database_path):
    subprocess.call(["mdb-schema", database_path, "mysql"])
    # Get the list of table names with "mdb-tables"
    table_names = subprocess.Popen(["mdb-tables", "-1", database_path],
                                   stdout=subprocess.PIPE).communicate()[0]
    tables = table_names.splitlines()
    sys.stdout.flush()

    # Dump each table as a stringio using "mdb-export",
    out_tables = {}
    for rtable in tables:
        table = rtable.decode()
        if verbose: print('running table:',table)
        if table != '':
            if verbose: print("Dumping " + table)
            contents = subprocess.Popen(["mdb-export", database_path, table],
                                        stdout=subprocess.PIPE).communicate()[0]
            temp_io = StringIO(contents.decode())
            print(table, temp_io)
            out_tables[table] = pd.read_csv(temp_io)
    return out_tables

# %% Merge staff qualification tables for 2015-2019
def clean_2015_2017_data(df, year):
    df.rename({'SCHOOL_NAME': 'ENTITY_NAME'}, axis='columns', inplace=True)
    df.rename({'NUM_FEWER_3YRS_EXP': 'NUM_TEACH_INEXP'}, axis='columns', inplace=True)
    df.rename({'NUM_TEACH_OUT_CERT': 'NUM_OUT_CERT'}, axis='columns', inplace=True)
    df = df[['ENTITY_CD', 'NUM_TEACH', 'NUM_TEACH_INEXP', 'NUM_OUT_CERT']]
    df = df.groupby(['ENTITY_CD']).sum()
    df['YEAR'] = year
    df['PER_TEACH_INEXP'] = 100 * df['NUM_TEACH_INEXP'] / df['NUM_TEACH']
    df['PER_OUT_CERT'] = 100 * df['NUM_OUT_CERT'] / df['NUM_TEACH']
    df.update(df[['PER_TEACH_INEXP','PER_OUT_CERT']].fillna(0))
    df = df.round(decimals=2)
    return df

def clean_2018_2019_data(df, year):
    df = df[['ENTITY_CD', 'NUM_TEACH', 'NUM_TEACH_INEXP', 'NUM_TEACH_OC', 'NUM_OUT_CERT']]
    df = df.groupby(['ENTITY_CD']).sum()
    df['YEAR'] = year
    df['PER_TEACH_INEXP'] = 100 * df['NUM_TEACH_INEXP'] / df['NUM_TEACH']
    df['PER_OUT_CERT'] = 100 * df['NUM_OUT_CERT'] / df['NUM_TEACH_OC']
    df.update(df[['PER_TEACH_INEXP','PER_OUT_CERT']].fillna(0))
    df.drop(['NUM_TEACH_OC'], axis=1, inplace=True)
    df = df.round(decimals=2)
    return df


In [3]:
filepath = '../Data/df_staff_qualifications_19_20.pkl'
df_staff_qualifications_19_20 = pd.read_pickle(filepath)

In [4]:
# Query tables
"""
# Graduation Rate and Outcomes 2019
database_path = 'gradrate_2019/GRAD_RATE_AND_OUTCOMES_2019.mdb'
out_tables = mdb_to_pandas(database_path)
table_names = list(out_tables.keys())
df_grad_rate_19 = out_tables['GRAD_RATE_AND_OUTCOMES_2019']

print('-------')
print('\ntable names = ', table_names)
print('\ntable columns = ', df_grad_rate_19.columns)
print('\ntable head = ', df_grad_rate_19.head())
print('unique nrc_desc', df_grad_rate_19['nrc_desc'].unique())
print('unique subgroup_name', df_grad_rate_19['subgroup_name'])
"""
"""
# Graduation Rate and Outcomes 2018
database_path = 'gradrate_2018/GRAD_RATE_AND_OUTCOMES_2018.mdb'
out_tables = mdb_to_pandas(database_path)
table_names = list(out_tables.keys())
df_grad_rate_18 = out_tables['GRAD_RATE_AND_OUTCOMES_2018']

print('-------')
print('\ntable names = ', table_names)
print('\ntable columns = ', df_grad_rate_18.columns)
print('\ntable head = ', df_grad_rate_18.head())

# Graduation Rate and Outcomes 2017
database_path = 'gradrate_2017/GRAD_RATE_AND_OUTCOMES_2017.mdb'
out_tables = mdb_to_pandas(database_path)
table_names = list(out_tables.keys())
df_grad_rate_17 = out_tables['GRAD_RATE_AND_OUTCOMES_2017']

print('-------')
print('\ntable names = ', table_names)
print('\ntable columns = ', df_grad_rate_17.columns)
print('\ntable head = ', df_grad_rate_17.head())

# Graduation Rate and Outcomes 2016
database_path = 'gradrate_2016/GRAD_RATE_AND_OUTCOMES_2016.mdb'
out_tables = mdb_to_pandas(database_path)
table_names = list(out_tables.keys())
df_grad_rate_16 = out_tables['GRAD_RATE_AND_OUTCOMES_2016']

print('-------')
print('\ntable names = ', table_names)
print('\ntable columns = ', df_grad_rate_16.columns)
print('\ntable head = ', df_grad_rate_16.head())

# Graduation Rate and Outcomes 2015
database_path = 'gradrate_2015/GRAD_RATE_AND_OUTCOMES_2015.mdb'
out_tables = mdb_to_pandas(database_path)
table_names = list(out_tables.keys())
df_grad_rate_15 = out_tables['GRAD_RATE_AND_OUTCOMES_2015']

print('-------')
print('\ntable names = ', table_names)
print('\ntable columns = ', df_grad_rate_15.columns)
print('\ntable head = ', df_grad_rate_15.head())


# Report Card Staff Qualifications 2019
database_path = '../Data/SRC2019_20200703.mdb'
out_tables = mdb_to_pandas(database_path)
table_names = list(out_tables.keys())
df_staff_qualifications_19_20 = out_tables['Staff Qualifications']

print('-------')
print('\ntable names = ', table_names)
print('\ntable columns = ', df_staff_qualifications_19_20.columns)
print('\ntable head = ', df_staff_qualifications_19_20.head())

df_staff_qualifications_19_20.to_pickle('../Data/df_staff_qualifications_19_20.pkl')

# Report Card Staff Qualifications 2018
database_path = 'SRC2018/SRC2018_20190627.mdb'
out_tables = mdb_to_pandas(database_path)
table_names = list(out_tables.keys())
df_staff_qualifications_18_19 = out_tables['Staff Qualifications']

print('-------')
print('\ntable names = ', table_names)
print('\ntable columns = ', df_staff_qualifications_18_19.columns)
print('\ntable head = ', df_staff_qualifications_18_19.head())

# Report Card Staff Qualifications 2017
database_path = 'SRC2017/SRC2017GroupIIRelease.mdb'
out_tables = mdb_to_pandas(database_path)
table_names = list(out_tables.keys())
df_staff_qualifications_17 = out_tables['Staff']

print('-------')
print('\ntable names = ', table_names)
print('\ntable columns = ', df_staff_qualifications_17.columns)
print('\ntable head = ', df_staff_qualifications_17.head())

# Report Card Staff Qualifications 2016
database_path = 'SRC2016/SRC2016_GroupIII.mdb'
out_tables = mdb_to_pandas(database_path)
table_names = list(out_tables.keys())
df_staff_qualifications_16 = out_tables['Staff']

print('-------')
print('\ntable names = ', table_names)
print('\ntable columns = ', df_staff_qualifications_16.columns)
print('\ntable head = ', df_staff_qualifications_16.head())

# Report Card Staff Qualifications 2015
database_path = 'SRC2015/SRC2015.mdb'
out_tables = mdb_to_pandas(database_path)
table_names = list(out_tables.keys())
df_staff_qualifications_15 = out_tables['Staff']

print('-------')
print('\ntable names = ', table_names)
print('\ntable columns = ', df_staff_qualifications_15.columns)
print('\ntable head = ', df_staff_qualifications_15.head())
"""

"\n# Graduation Rate and Outcomes 2018\ndatabase_path = 'gradrate_2018/GRAD_RATE_AND_OUTCOMES_2018.mdb'\nout_tables = mdb_to_pandas(database_path)\ntable_names = list(out_tables.keys())\ndf_grad_rate_18 = out_tables['GRAD_RATE_AND_OUTCOMES_2018']\n\nprint('-------')\nprint('\ntable names = ', table_names)\nprint('\ntable columns = ', df_grad_rate_18.columns)\nprint('\ntable head = ', df_grad_rate_18.head())\n\n# Graduation Rate and Outcomes 2017\ndatabase_path = 'gradrate_2017/GRAD_RATE_AND_OUTCOMES_2017.mdb'\nout_tables = mdb_to_pandas(database_path)\ntable_names = list(out_tables.keys())\ndf_grad_rate_17 = out_tables['GRAD_RATE_AND_OUTCOMES_2017']\n\nprint('-------')\nprint('\ntable names = ', table_names)\nprint('\ntable columns = ', df_grad_rate_17.columns)\nprint('\ntable head = ', df_grad_rate_17.head())\n\n# Graduation Rate and Outcomes 2016\ndatabase_path = 'gradrate_2016/GRAD_RATE_AND_OUTCOMES_2016.mdb'\nout_tables = mdb_to_pandas(database_path)\ntable_names = list(out_tables.

In [5]:
%%time
# Report Card Staff Qualifications 2019
database_path = '../Data/SRC2019_20200703.mdb'
out_tables = mdb_to_pandas(database_path)
table_names = list(out_tables.keys())

ACC EM Chronic Absenteeism <_io.StringIO object at 0x117e51b90>


/Users/dave/src/CSE6242/.venv/lib/python3.7/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


ACC EM Combined Composite Growth <_io.StringIO object at 0x117e515f0>
ACC EM Composite Performance <_io.StringIO object at 0x117e51870>


/Users/dave/src/CSE6242/.venv/lib/python3.7/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (5,7,8,10,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


ACC EM ELP <_io.StringIO object at 0x117e51a50>


...

ACC HS Participation Rate <_io.StringIO object at 0x117e517d0>
ACC HS Progress <_io.StringIO object at 0x117e51af0>


/Users/dave/src/CSE6242/.venv/lib/python3.7/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


Accountability Status <_io.StringIO object at 0x11b0e9230>
Accountability Status by Subgroup <_io.StringIO object at 0x11b0e92d0>
Annual EM ELA <_io.StringIO object at 0x11b0e9190>
Annual EM MATH <_io.StringIO object at 0x11b0e94b0>
Annual EM SCIENCE <_io.StringIO object at 0x11b0e9550>
Annual NYSAA <_io.StringIO object at 0x11b0e95f0>
Annual NYSESLAT <_io.StringIO object at 0x11b0e94b0>
BOCES and N/RC <_io.StringIO object at 0x11b0e9730>
Expenditures per Pupil <_io.StringIO object at 0x11b0e97d0>
Institution Grouping <_io.StringIO object at 0x11b0e95f0>
Staff Qualifications <_io.StringIO object at 0x11b0e9870>
Total Cohort Regents Exams <_io.StringIO object at 0x11b0e9910>
ACC EM Recently Arrived ELLs <_io.StringIO object at 0x11b0e9a50>
Accountability Levels <_io.StringIO object at 0x11b0e9870>


/Users/dave/src/CSE6242/.venv/lib/python3.7/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


Annual Regents Exams <_io.StringIO object at 0x11b0e9370>
CPU times: user 9.67 s, sys: 3.98 s, total: 13.6 s
Wall time: 3min 40s


## Tables List
Here are the tables in the 2019 data. Anything table that contains data types that were note cleanly inferred (i.e. has columns with mixed data types) are notated with a hyphen and the word "Mixed".

* ACC EM Chronic Absenteeism - Mixed 
# ACC EM Combined Composite Growth 
* ACC EM Composite Performance - Mixed
* ACC EM ELP - Mixed
* ACC EM Growth - Mixed
* ACC EM Participation Rate
* ACC EM Progress- Mixed
* ACC HS CCCR
* ACC HS Chronic Absenteeism 
* ACC HS Combined Composite Grad 
* ACC HS Composite Performance - Mixed
* ACC HS ELP 
* ACC HS Graduation Rate - Mixed
* ACC HS Participation Rate 
* ACC HS Progress - Mixed
* Accountability Status 
* Accountability Status by Subgroup 
* Annual EM ELA 
* Annual EM MATH 
* Annual EM SCIENCE 
* Annual NYSAA 
* Annual NYSESLAT 
* BOCES and N/RC 
* Expenditures per Pupil 
* Institution Grouping 
* Staff Qualifications 
* Total Cohort Regents Exams 
* ACC EM Recently Arrived ELLs 
* Accountability Levels - Mixed
* Annual Regents Exams

In [15]:
years = ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',\
         '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']

file_names = out_tables.keys()

for year in years:
    mdb_name = "SRC" + year + ".mdb"
    print(mdb_name)
    

SRC2000.mdb
SRC2001.mdb
SRC2002.mdb
SRC2003.mdb
SRC2004.mdb
SRC2005.mdb
SRC2006.mdb
SRC2007.mdb
SRC2008.mdb
SRC2009.mdb
SRC2010.mdb
SRC2011.mdb
SRC2012.mdb
SRC2013.mdb
SRC2014.mdb
SRC2015.mdb
SRC2016.mdb
SRC2017.mdb
SRC2018.mdb
SRC2019.mdb


In [9]:

df_staff_qualifications_19_20 = out_tables['Staff Qualifications']

print('-------')
print('\ntable names = ', table_names)
print('\ntable columns = ', df_staff_qualifications_19_20.columns)
print('\ntable head = ', df_staff_qualifications_19_20.head())

df_staff_qualifications_19_20.to_pickle('../Data/df_staff_qualifications_19_20.pkl')

KeyError: 'Staff Qualifications'

In [16]:
# Report Card Staff Qualifications 2018
database_path = 'SRC2018/SRC2018_20190627.mdb'
out_tables = mdb_to_pandas(database_path)
table_names = list(out_tables.keys())
df_staff_qualifications_18_19 = out_tables['Staff Qualifications']

print('-------')
print('\ntable names = ', table_names)
print('\ntable columns = ', df_staff_qualifications_18_19.columns)
print('\ntable head = ', df_staff_qualifications_18_19.head())

NameError: name 'df_staff_qualifications' is not defined

In [ ]:
# Report Card Staff Qualifications 2017
database_path = 'SRC2017/SRC2017GroupIIRelease.mdb'
out_tables = mdb_to_pandas(database_path)
table_names = list(out_tables.keys())
df_staff_qualifications_17 = out_tables['Staff']

print('-------')
print('\ntable names = ', table_names)
print('\ntable columns = ', df_staff_qualifications_17.columns)
print('\ntable head = ', df_staff_qualifications_17.head())

In [ ]:
# Report Card Staff Qualifications 2016
database_path = 'SRC2016/SRC2016_GroupIII.mdb'
out_tables = mdb_to_pandas(database_path)
table_names = list(out_tables.keys())
df_staff_qualifications_16 = out_tables['Staff']

print('-------')
print('\ntable names = ', table_names)
print('\ntable columns = ', df_staff_qualifications_16.columns)
print('\ntable head = ', df_staff_qualifications_16.head())

In [18]:
# Report Card Staff Qualifications 2015
database_path = 'SRC2015/SRC2015.mdb'
out_tables = mdb_to_pandas(database_path)
table_names = list(out_tables.keys())
df_staff_qualifications_15 = out_tables['Staff']

print('-------')
print('\ntable names = ', table_names)
print('\ntable columns = ', df_staff_qualifications_15.columns)
print('\ntable head = ', df_staff_qualifications_15.head())

KeyError: 'Staff'